In [1]:
import sys
sys.path.append("..")
from webapp.utils.azure_utils import KeyVault, DataLake
from webapp.utils.data_pre_utils import df_val_map
import numpy as np
import pandas as pd
import datetime
import fastparquet
import os

In [2]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)

In [3]:
file_system = "energyhub"
dest_dir = "/data_parq/norm_data"
meters_dir = "/data_parq/meters"
meta_dir = "/data_parq/metadata"
weather_dir = "/data_parq/weather"


In [4]:
meter = storage.read(file_system, meters_dir, file_name="electricity.parq", extension="parq")


In [5]:
meta = storage.read(file_system, meta_dir, file_name="metadata.parq", extension="parq")


In [6]:
weather = storage.read(file_system, weather_dir, file_name="weather.parq", extension="parq")

In [7]:
meter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27684432 entries, 0 to 27684431
Data columns (total 3 columns):
 #   Column       Dtype  
---  ------       -----  
 0   timestamp    object 
 1   building_id  object 
 2   electricity  float64
dtypes: float64(1), object(2)
memory usage: 633.6+ MB


In [8]:
meter.isna().sum()*100/len(meter)

timestamp      0.000000
building_id    0.000000
electricity    8.928675
dtype: float64

In [9]:
# number of buildings in dataset
len(meter.building_id.unique())

1578

In [10]:
meter.groupby("building_id").count().rsub(meter.groupby("building_id").size(), axis=0)

,timestamp,electricity
building_id,,
Bear_assembly_Angel,0,3109
Bear_assembly_Beatrice,0,39
Bear_assembly_Danial,0,5357
Bear_assembly_Diana,0,2266
Bear_assembly_Genia,0,167
...,...,...
Wolf_public_Norma,0,1
Wolf_retail_Harriett,0,484
Wolf_retail_Marcella,0,1


In [11]:
meter[meter["electricity"].isna()].groupby("building_id").count()

,timestamp,electricity
building_id,,
Bear_assembly_Angel,3109,0
Bear_assembly_Beatrice,39,0
Bear_assembly_Danial,5357,0
Bear_assembly_Diana,2266,0
Bear_assembly_Genia,167,0
...,...,...
Wolf_public_Norma,1,0
Wolf_retail_Harriett,484,0
Wolf_retail_Marcella,1,0


In [12]:
meta.columns

Index(['building_id', 'site_id', 'building_id_kaggle', 'site_id_kaggle',
       'primary_space_usage', 'sub_primary_space_usage', 'sq_meter', 'sq_feet',
       'latitude', 'longitude', 'timezone', 'electricity', 'hotwater',
       'chilledwater', 'steam', 'water', 'irrigation', 'solar', 'gas',
       'industry', 'subindustry', 'heating_type', 'year_built', 'date_opened',
       'number_of_floors', 'occupants', 'energy_stars_core', 'eui', 'site_eui',
       'source_eui', 'leed_level', 'rating'],
      dtype='object')

In [13]:
meta.isna().sum()*100/len(meta)

building_id                  0.000000
site_id                      0.000000
building_id_kaggle          11.430318
site_id_kaggle               2.261614
primary_space_usage          1.283619
sub_primary_space_usage      1.283619
sq_meter                     0.000000
sq_feet                      0.000000
latitude                    14.486553
longitude                   14.486553
timezone                     0.000000
electricity                100.000000
hotwater                     0.000000
chilledwater                 0.000000
steam                        0.000000
water                        0.000000
irrigation                   0.000000
solar                        0.000000
gas                          0.000000
industry                    64.608802
subindustry                 64.608802
heating_type                86.858191
year_built                  50.061125
date_opened                 98.716381
number_of_floors            73.044010
occupants                   85.941320
energy_stars

In [14]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331166 entries, 0 to 331165
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   timestamp                 331166 non-null  object 
 1   site_id                   331166 non-null  object 
 2   air_temperature           331038 non-null  float64
 3   cloud_coverage            160179 non-null  float64
 4   dew_temperature           330838 non-null  float64
 5   precipitation_depth_1_hr  197980 non-null  float64
 6   precipitation_depth_6_hr  18162 non-null   float64
 7   sea_level_pressure        309542 non-null  float64
 8   wind_direction            318161 non-null  float64
 9   wind_speed                330592 non-null  float64
dtypes: float64(8), object(2)
memory usage: 25.3+ MB


In [15]:
weather.isna().sum()*100/len(weather)

timestamp                    0.000000
site_id                      0.000000
air_temperature              0.038651
cloud_coverage              51.631810
dew_temperature              0.099044
precipitation_depth_1_hr    40.217293
precipitation_depth_6_hr    94.515741
sea_level_pressure           6.529656
wind_direction               3.927034
wind_speed                   0.173327
dtype: float64

## Process Datasets

In [16]:
def process_meter(meter_df):
    #drop NA
    meter_df.dropna(inplace=True)
    #round
    meter_df["electricity"] = meter_df['electricity'].astype(float).round(4)
    meter_df["timestamp"] = pd.to_datetime(meter_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    meter_df["month"] = meter_df["timestamp"].dt.month
    meter_df["weekday"] = meter_df["timestamp"].dt.weekday
    meter_df["hour"] = meter_df["timestamp"].dt.hour
    #maybe add holiday here too?
    return meter_df

In [17]:
def process_meta(meta_df):
    #use these cols because they have low % of NA values. Other cols are mostly NA
    use_cols = ['building_id', 'site_id', 'primary_space_usage', 'sq_meter']
    drop_cols = [col for col in meta_df.columns if col not in use_cols]
    meta_df.drop(columns=drop_cols, inplace=True)
    meta_df['primary_space_usage'].fillna("Unknown", inplace=True )
    return meta_df

In [18]:
def process_weather(weather_df):
    weather_df.drop(columns = ['cloud_coverage','precipitation_depth_1_hr','precipitation_depth_6_hr','sea_level_pressure'], inplace=True)
    weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    weather_df['air_temperature'].interpolate(method="linear", inplace=True)
    weather_df['dew_temperature'].interpolate(method="linear", inplace=True)
    weather_df['wind_direction'].interpolate(method="linear", inplace=True)
    weather_df['wind_speed'].interpolate(method="linear", inplace=True)
    return weather_df


In [19]:
meter_df = process_meter(meter)

In [20]:
meta_df = process_meta(meta)

In [21]:
weather_df = process_weather(weather)

In [22]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331166 entries, 0 to 331165
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   timestamp        331166 non-null  datetime64[ns]
 1   site_id          331166 non-null  object        
 2   air_temperature  331166 non-null  float64       
 3   dew_temperature  331166 non-null  float64       
 4   wind_direction   331166 non-null  float64       
 5   wind_speed       331166 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 15.2+ MB


In [23]:
meter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25212579 entries, 747 to 27676291
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   timestamp    datetime64[ns]
 1   building_id  object        
 2   electricity  float64       
 3   month        int64         
 4   weekday      int64         
 5   hour         int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 1.3+ GB


In [24]:
df = meter_df.merge(meta_df, how="left", on="building_id")\
            .merge(weather_df, how="left", on = ["timestamp", "site_id"])

In [25]:
df.head()

,timestamp,building_id,electricity,month,weekday,hour,site_id,primary_space_usage,sq_meter,air_temperature,dew_temperature,wind_direction,wind_speed
0,2016-02-01 03:00:00,Panther_parking_Lorriane,3.7207,2,0,3,Panther,Parking,36012.7,17.2,15.0,0.0,0.0
1,2016-02-06 13:00:00,Panther_parking_Lorriane,2.1004,2,5,13,Panther,Parking,36012.7,17.8,12.8,20.0,4.1
2,2016-02-14 05:00:00,Panther_parking_Lorriane,3.3006,2,6,5,Panther,Parking,36012.7,6.7,3.3,350.0,3.6
3,2016-02-20 01:00:00,Panther_parking_Lorriane,3.4807,2,5,1,Panther,Parking,36012.7,13.3,10.0,10.0,1.5
4,2016-02-29 11:00:00,Panther_parking_Lorriane,2.1604,2,0,11,Panther,Parking,36012.7,21.7,8.3,100.0,1.5


In [26]:
df.isna().sum()*100/len(df)

timestamp              0.000000
building_id            0.000000
electricity            0.000000
month                  0.000000
weekday                0.000000
hour                   0.000000
site_id                0.000000
primary_space_usage    0.000000
sq_meter               0.000000
air_temperature        0.337066
dew_temperature        0.337066
wind_direction         0.337066
wind_speed             0.337066
dtype: float64

In [28]:
df['air_temperature'].interpolate(method="linear", inplace=True)
df['air_temperature'] = df['air_temperature'].astype(float).round(2)
df['dew_temperature'].interpolate(method="linear", inplace=True)
df['dew_temperature'] = df['dew_temperature'].astype(float).round(2)
df['wind_direction'].interpolate(method="linear", inplace=True)
df['wind_direction'] = df['wind_direction'].astype(float).round(2)
df['wind_speed'].interpolate(method="linear", inplace=True)
df['wind_speed'] =df['wind_speed'].astype(float).round(2)

In [29]:
df.isna().sum()*100/len(df)

timestamp              0.0
building_id            0.0
electricity            0.0
month                  0.0
weekday                0.0
hour                   0.0
site_id                0.0
primary_space_usage    0.0
sq_meter               0.0
air_temperature        0.0
dew_temperature        0.0
wind_direction         0.0
wind_speed             0.0
dtype: float64

In [30]:
file = "electricity_features.parq"
#change to parquet file
fastparquet.write(file, df, compression="GZIP")

Error in sys.excepthook:
Traceback (most recent call last):
  File "c:\Users\phuon\Documents\Projects\project\venv\lib\site-packages\IPython\core\interactiveshell.py", line 1934, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OverflowError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\phuon\Documents\Projects\project\venv\lib\site-packages\IPython\core\interactiveshell.py", line 1936, in showtraceback
    stb = self.InteractiveTB.structured_traceback(etype,
  File "c:\Users\phuon\Documents\Projects\project\venv\lib\site-packages\IPython\core\ultratb.py", line 1105, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\phuon\Documents\Projects\project\venv\lib\site-packages\IPython\core\ultratb.py", line 999, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\Users\phuon\Documents\Proj

In [31]:
#write to azure
storage.upload(file_system=file_system, directory=dest_dir, file_name=file,file_path=file,overwrite=True )
os.remove(file)

electricity_features.parq write complete
